%pip install --upgrade astrapy

%pip install -q cassandra-driver
%pip install -q langchain
%pip install -q chat-deepseek-api
%pip install -q pypdf
%pip install -q cassio>=0.1.1

%pip install cassandra-driver

In [10]:
import cassandra

In [2]:
from cassandra.cluster import Cluster 

In [3]:
from cassandra.auth import PlainTextAuthProvider
import json

cloud_config= {'secure_connect_bundle':'secure-connect-pdf-qa (1).zip'}

with open("D:\PROJECTS\RagProject\yukkualahari@gmail.com-token (1).json") as f:
    secrets = json.load(f)
    
CLIENT_ID=secrets['clientId']
CLIENT_SECRET=secrets['secret']

auth_provider = PlainTextAuthProvider(CLIENT_ID, CLIENT_SECRET)
cluster = Cluster(cloud=cloud_config, auth_provider=auth_provider)
session = cluster.connect()

In [4]:
row = session.execute("select release_version from system.local").one()

if row:
    print(row[0])
else:
    print("An error occurred.")

4.0.11-7cde36df13c4


%pip install langchain-deepseek-official langchain-community

In [12]:
from langchain_deepseek import ChatDeepSeek
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores.cassandra import Cassandra
from langchain.indexes import VectorstoreIndexCreator
from langchain.text_splitter import CharacterTextSplitter
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.document_loaders import TextLoader, PyPDFLoader
from langchain.docstore.document import Document
from langchain.chains import ConversationalRetrievalChain

%pip install langchain-openai

%pip install openrouter-python langchain-community

In [4]:
from langchain_community.llms import Ollama

In [5]:
llm = Ollama(
    base_url="http://localhost:11434",
    model="deepseek-r1:1.5b",
    temperature=0.3,
    num_gpu=1,  # Use GPU acceleration
    num_predict=128,
    num_ctx=800  # Use 1024 context tokens
)

C:\Users\yukku\AppData\Local\Temp\ipykernel_9700\392549622.py:1: LangChainDeprecationWarning: The class `Ollama` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import OllamaLLM``.
  llm = Ollama(


In [6]:
from langchain_community.embeddings import HuggingFaceEmbeddings

embeddings = HuggingFaceEmbeddings(
    model_name="sentence-transformers/all-mpnet-base-v2"
)


C:\Users\yukku\AppData\Local\Temp\ipykernel_9700\2996441843.py:3: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(
d:\PROJECTS\RagProject\Rag\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [10]:
loader = PyPDFLoader("D:\PROJECTS\RagProject\how-to-win-friends-and-influence-people.pdf")
pages = loader.load_and_split()

In [11]:
len(pages)

192

In [13]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=400,
    chunk_overlap=30
)

In [7]:
keyspace = "pdf_qa_name"  
table_name = "pdf_qa_table"

In [13]:
vectorstore = Cassandra(
    embedding=embeddings,
    session=session,
    keyspace=keyspace,
    table_name=table_name
)

In [20]:
texts = text_splitter.split_documents(pages)
vectorstore.add_documents(texts)

['125342c15dd7474d942c85cffda49604',
 '40fe1210014847bab1d1fb240abacf74',
 'df5c4bc5d9914bdea9c27f785cbd7099',
 'd7bd7097672741729b552d4de856cf0f',
 '2241ce80e0ac480b8885fbfd2fc5bf03',
 '13bb6b842a674b93bd0181a48638b2fe',
 '8a817ead38e24b189aee27f0f2d56844',
 '939d96fc682546dda419d36932906d9f',
 'e6857f9c086346619ea8624f645f20ad',
 'ccc0acd07325459f891065f236831606',
 'f3fd89c33120482e922777ba60097f33',
 'cd2543bb2233452493ef6e543ebbf036',
 '74c08946a8a44dd8be14ea251a532bde',
 '0283409da2e14392bf646890db9b9fa7',
 'e52b1546298c42f7a55e2e551df01fe5',
 'b644917c3e954875ac1f4ef86437b8e1',
 'fd13e15e3adc4c32acc5c7c0692c25c5',
 '5f9426fa91a943ccb07138850a8c4f91',
 'd3c27afb856c45af9faddf70f74a7752',
 'ad326edd06ce4138b50669adaa4e54b0',
 'e919616253e74b92b11c81844cf4b93b',
 '5adb90be42294e28945768bb45119eaf',
 '007473802a5a4d11bfdcd76d418c06d5',
 '9e3932b3b8d24642be777f8e318e9235',
 '3f081570b5724dd7aa8c4a4e35078882',
 'c73d389c49fb45629654b77d795514cd',
 '1d17ac293e614d0cb8c2907780c727fb',
 

In [18]:
default_query = f"SELECT * FROM {keyspace}.{table_name}"
rows = session.execute(default_query)

In [ ]:
for idx, row in enumerate(rows):
    print(f"\nRow [{idx}]")
    print(f"row_id: {row.row_id}")
    print(f"embedding_vector: {str(row.vector)[:64]} ...")


Row [0]
row_id: dcae5eb545aa4385ab4d976277698dbd
embedding_vector: [0.0004363643529359251, 0.1571173071861267, -0.01136789098381996 ...

Row [1]
row_id: c4157966ee244feb97660362e1e5905d
embedding_vector: [0.02411539852619171, 0.08131307363510132, 0.023404929786920547, ...

Row [2]
row_id: 7d17b2c1c689457482c8b0298128b2b2
embedding_vector: [-0.017383994534611702, 0.029291825369000435, 0.0241816509515047 ...

Row [3]
row_id: c5757cbca68b45fa92609eeb56822d7b
embedding_vector: [-0.030268263071775436, 0.021602487191557884, -0.015518531203269 ...

Row [4]
row_id: 873b84ebd7f04c70b0713e14ff4ddb1b
embedding_vector: [0.051442742347717285, 0.041276656091213226, -0.0076317535713315 ...

Row [5]
row_id: b028c8f071504c3bba257b0844cbfd4c
embedding_vector: [0.013924135826528072, 0.04837240278720856, 0.009133261628448963 ...

Row [6]
row_id: 0ca13fc9d8a943d38504ad8ba90888a5
embedding_vector: [0.049271129071712494, 0.11985959112644196, 0.01827067695558071, ...

Row [7]
row_id: f1e6d8a0ddce4b0eaed308ef

In [19]:
qa_chain = ConversationalRetrievalChain.from_llm(
    llm=llm,  # Only this line changes from previous OpenAI setup
    retriever=vectorstore.as_retriever(),
    return_source_documents=True
)

[Your PDFs] → [Cassandra Vector Store (unchanged)] 
                     ↓
[Local DeepSeek LLM] → [Conversational Chain (modified)]
                     ↑
           [Docker Container (ollama:11434)]

In [20]:
chat_history = []

In [21]:
query = "Can you analyze the document and explain what is the big secret of dealing with people?"
result = qa_chain({"question": query, "chat_history": chat_history})
chat_history.append(result)

C:\Users\yukku\AppData\Local\Temp\ipykernel_9700\1145979245.py:2: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  result = qa_chain({"question": query, "chat_history": chat_history})


OllamaEndpointNotFoundError: Ollama call failed with status code 404. Maybe your model is not found and you should pull the model with `ollama pull deepseek-r1:1.5b`.

In [ ]:
print(result["answer"])

In order to get the most out of this book, devote a deep, driving desire to mastering the principles of human relations. By keeping a record of specific examples of how to apply these principles in practice, you will be inspired and motivated to make greater efforts in future endeavors. Additionally, by developing a strong desire to learn and improve your skills, you will be able to more effectively communicate with others and achieve better results in all areas of life.
